# Experiment

## Install Python dependencies

In [1]:
!pip install -q onnx onnxruntime tf2onnx

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mypy-protobuf 3.6.0 requires protobuf>=4.25.3, but you have protobuf 3.20.3 which is incompatible.
feast 0.40.1 requires protobuf<5.0.0,>=4.24.0, but you have protobuf 3.20.3 which is incompatible.
codeflare-sdk 0.19.1 requires pydantic<2, but you have pydantic 2.9.2 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


Import the dependencies for the model training code:

In [2]:
import datetime
import numpy as np
import onnx
import pandas as pd
import pickle
import tf2onnx

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from pathlib import Path

2024-09-25 15:29:05.634317: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 15:29:05.634398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 15:29:05.635805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 15:29:05.643059: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 15:29:06.576176: W tensorflow/compiler/tf2

The output might show TensorFlow messages, such as a "Could not find TensorRT" warning. You can ignore these messages.


## Load the CSV data

The CSV data that you use to train the model contains the following fields:

* **distancefromhome** - The distance from home where the transaction happened.
* **distancefromlast_transaction** - The distance from the last transaction that happened.
* **ratiotomedianpurchaseprice** - The ratio of purchased price compared to median purchase price.
* **repeat_retailer** - If it's from a retailer that already has been purchased from before.
* **used_chip** - If the credit card chip was used.
* **usedpinnumber** - If the PIN number was used.
* **online_order** - If it was an online order.
* **fraud** - If the transaction is fraudulent.

## Install Feast project

In [3]:
!pip install -q feast
!feast version

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf2onnx 1.16.1 requires protobuf~=3.20, but you have protobuf 4.25.5 which is incompatible.
kfp 2.5.0 requires protobuf<4,>=3.13.0, but you have protobuf 4.25.5 which is incompatible.
kfp-pipeline-spec 0.2.2 requires protobuf<4,>=3.13.0, but you have protobuf 4.25.5 which is incompatible.
kfp-kubernetes 1.0.0 requires protobuf<4,>=3.13.0, but you have protobuf 4.25.5 which is incompatible.
codeflare-sdk 0.19.1 requires pydantic<2, but you have pydantic 2.9.2 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Feast SDK Version: "0.40.1"


In [4]:
# Forward Feastlogs to the notebook output
import logging
import sys
from io import StringIO
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

In [5]:
%env FEAST_REPO=feast_fraud/feature_repo
%env ROOT_DIR=/opt/app-root/src/fraud-detection
!ls $FEAST_REPO

env: FEAST_REPO=feast_fraud/feature_repo
env: ROOT_DIR=/opt/app-root/src/fraud-detection
data  feature_store.yaml


In [6]:
!feast -c $FEAST_REPO apply
!feast -c $FEAST_REPO entities list
!feast -c $FEAST_REPO data-sources list
!feast -c $FEAST_REPO feature-views list

Deleted entity customer
Deleted feature view validation
Deleted feature view training
Deleted feature view test

No changes to infrastructure
NAME    DESCRIPTION    TYPE
NAME    CLASS
NAME    ENTITIES    TYPE


In [7]:
# common imports
from datetime import timedelta

import pandas as pd
import os

from feast import (
    Entity,
    FeatureService,
    FeatureView,
    Field,
    FileSource,
    PushSource,
    RequestSource,
)
from feast.feature_store import FeatureStore
from feast.feature_logging import LoggingConfig
from feast.infra.offline_stores.file_source import FileLoggingDestination
from feast.on_demand_feature_view import on_demand_feature_view
from feast.types import Float32, Float64, Int64

In [8]:
# Initialize the store
store = FeatureStore(os.environ['FEAST_REPO'])
print(store.list_entities())

2024-09-25 15:29:32,417 - Registry cache expired, so refreshing
2024-09-25 15:29:32,420 - Registry cache expired, so refreshing


[]


## Data preparation
1. Add entity key to the tables
1. Add event_ts field
1. Convert to parquet files

Since the original dataset was not considering the `customer` concept, we'll replicate the same setup: add a new field customer_id and apply the same value to each record.

In [12]:
def add_customer_id(df):
    df['customer_id'] = 1

def add_timestamps(df):
    # Create time series: one entry every 1H, up to now
    timestamps = pd.date_range(
        end=pd.Timestamp.now().replace(microsecond=0), 
        periods=len(df), 
        freq='1H').to_frame(name="ts", index=False)

    timestamps['created'] = timestamps['ts']
    df = pd.concat(objs=[df, timestamps], axis=1)
    columns = df.columns.tolist()
    columns.insert(0, columns.pop(9))
    columns.insert(0, columns.pop(9))
    return df[columns]

In [13]:
xtrain = pd.read_csv('data/train.csv')
xval = pd.read_csv('data/validate.csv')
xtest = pd.read_csv('data/test.csv')

# add_customer_id(xtrain)
# add_customer_id(xval)
# add_customer_id(xtest)

xtrain = add_timestamps(xtrain)
xval = add_timestamps(xval)
xtest = add_timestamps(xtest)

!rm data/*.parquet
xtrain.to_parquet('data/train.parquet')
xval.to_parquet('data/validate.parquet')
xtest.to_parquet('data/test.parquet')

print("-----xtrain-----")
xtrain.info()
print("-----xval-----")
xval.info()
print("-----xtest-----")
xtest.info()

/tmp/ipykernel_949/1116142764.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range(
/tmp/ipykernel_949/1116142764.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range(
/tmp/ipykernel_949/1116142764.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range(


-----xtrain-----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ts                              600000 non-null  datetime64[ns]
 1   created                         600000 non-null  datetime64[ns]
 2   distance_from_home              600000 non-null  float64       
 3   distance_from_last_transaction  600000 non-null  float64       
 4   ratio_to_median_purchase_price  600000 non-null  float64       
 5   repeat_retailer                 600000 non-null  float64       
 6   used_chip                       600000 non-null  float64       
 7   used_pin_number                 600000 non-null  float64       
 8   online_order                    600000 non-null  float64       
 9   fraud                           600000 non-null  float64       
dtypes: datetime64[ns](2), float64(8)
memory

In [14]:
!ls -lh data/*parquet

-rw-r--r--. 1 1001210000 1001210000 8.8M Sep 25 15:29 data/test.parquet
-rw-r--r--. 1 1001210000 1001210000  24M Sep 25 15:29 data/train.parquet
-rw-r--r--. 1 1001210000 1001210000 8.8M Sep 25 15:29 data/validate.parquet


## Define a SQL store
A Postgres service is deployed on the current namespace and DB tables are created and populated with data from the `xtrain`, `xval` and `xtest` data frames

In [49]:
namespace_path='/var/run/secrets/kubernetes.io/serviceaccount/namespace'
with open(namespace_path, "r") as file:
    namespace = file.read().strip()
print(f"Current namespace is {namespace}")
os.environ['TARGET_NS'] = namespace
!echo $TARGET_NS

Current namespace is dmartino-fraud-detection
dmartino-fraud-detection


Create an instance of PostgreSQL database with:
DATABASE_SERVICE_NAME=postgresql 
POSTGRESQL_USER=feast 
POSTGRESQL_PASSWORD=feast
POSTGRESQL_DATABASE=feast 
VOLUME_CAPACITY=2Gi 
MEMORY_LIMIT=1Gi


In [75]:
import pandas as pd
# import psycopg2
# from sqlalchemy import create_engine

psqlHost = f'postgresql.{namespace}.svc.cluster.local'
psqlPort = 5432
psqlUsername = 'feast'
psqlPassword = 'feast'
psqlDb = 'feast'
psqlSchema = 'public'

In [ ]:
import psycopg
from sqlalchemy import create_engine
from sqlalchemy.exc import ProgrammingError
from sqlalchemy import text


engine = create_engine(f'postgresql+psycopg://{psqlUsername}:{psqlPassword}@{psqlHost}:{str(psqlPort)}/{psqlDb}')
print("Persisting xtrain...")
xtrain.to_sql('fraud_train', engine, if_exists='append', index=True, schema=psqlSchema)
print("Persisting xval...")
xval.to_sql('fraud_validate', engine, if_exists='append', index=True, schema=psqlSchema)
print("Persisting xtest...")
xtest.to_sql('fraud_test', engine, if_exists='append', index=True, schema=psqlSchema)

Persisting xtrain...
Persisting xval...
Persisting xtest...


## Define the Feature Store
* Map parquet files to `FileSource`s
* Define FeatureView for training purposes
....

In [15]:
# Create the FileSources
train_source = FileSource(
    name="train_source",
    path=f"{os.environ['ROOT_DIR']}/data/train.parquet",
    timestamp_field="ts",
    created_timestamp_column="created",
)
validate_source = FileSource(
    name="validate_source",
    path=f"{os.environ['ROOT_DIR']}/data/validate.parquet",
    timestamp_field="ts",
    created_timestamp_column="created",
)
test_source = FileSource(
    name="test_source",
    path=f"{os.environ['ROOT_DIR']}/data/test.parquet",
    timestamp_field="ts",
    created_timestamp_column="created",
)
store.registry.apply_data_source(train_source, store.project)
store.registry.apply_data_source(validate_source, store.project)
store.registry.apply_data_source(test_source, store.project)
!feast -c $FEAST_REPO data-sources list

NAME             CLASS
train_source     <class 'feast.infra.offline_stores.file_source.FileSource'>
validate_source  <class 'feast.infra.offline_stores.file_source.FileSource'>
test_source      <class 'feast.infra.offline_stores.file_source.FileSource'>


In [16]:
# Customer entity
# customer = Entity(name="customer", join_keys=["customer_id"])
# store.registry.apply_entity(customer, store.project)
!feast -c $FEAST_REPO entities list

NAME    DESCRIPTION    TYPE


In [42]:
training_fv = FeatureView(
    name="training",
    # entities=[customer],
    entities=[],
    ttl=timedelta(days=1),
    schema=[
        # Field(name="customer_id", dtype=Int64),
        Field(name="distance_from_last_transaction", dtype=Float64),
        Field(name="ratio_to_median_purchase_price", dtype=Float64),
        Field(name="used_chip", dtype=Float64),
        Field(name="used_pin_number", dtype=Float64),
        Field(name="online_order", dtype=Float64),
        Field(name="fraud", dtype=Float64),
    ],
    online=True,
    source=train_source,
    tags={"team": "training"},
)
validation_fv = FeatureView(
    name="validation",
    # entities=[customer],
    entities=[],
    ttl=timedelta(days=1),
    schema=[
        # Field(name="customer_id", dtype=Int64),
        Field(name="distance_from_last_transaction", dtype=Float64),
        Field(name="ratio_to_median_purchase_price", dtype=Float64),
        Field(name="used_chip", dtype=Float64),
        Field(name="used_pin_number", dtype=Float64),
        Field(name="online_order", dtype=Float64),
        Field(name="fraud", dtype=Float64),
    ],
    online=True,
    source=validate_source,
    tags={"team": "training"},
)
test_fv = FeatureView(
    name="test",
    # entities=[customer],
    entities=[],
    ttl=timedelta(days=1),
    schema=[
        # Field(name="customer_id", dtype=Int64),
        Field(name="distance_from_last_transaction", dtype=Float64),
        Field(name="ratio_to_median_purchase_price", dtype=Float64),
        Field(name="used_chip", dtype=Float64),
        Field(name="used_pin_number", dtype=Float64),
        Field(name="online_order", dtype=Float64),
        Field(name="fraud", dtype=Float64),
    ],
    online=True,
    source=test_source,
    tags={"team": "training"},
)
store.registry.apply_feature_view(training_fv, store.project)
store.registry.apply_feature_view(validation_fv, store.project)
store.registry.apply_feature_view(test_fv, store.project)
!feast -c $FEAST_REPO feature-views list

/opt/app-root/lib64/python3.9/site-packages/feast/feature_view.py:425: UserWarning: There are some mismatches in your feature view's registered entities. Please check if you have applied your entities correctly.Entities: ['__dummy'] vs Entity Columns: []
  warnings.warn(


/opt/app-root/lib64/python3.9/site-packages/feast/feature_view.py:425: UserWarning: There are some mismatches in your feature view's registered entities. Please check if you have applied your entities correctly.Entities: ['__dummy'] vs Entity Columns: []
  warnings.warn(
NAME        ENTITIES    TYPE
training    n/a         FeatureView
validation  n/a         FeatureView
test        n/a         FeatureView


## Model training

In [18]:
# import subprocess

# # Run feast serve in the background
# feast_offline_server_process = subprocess.Popen(["feast", "-c", os.environ['FEAST_REPO'], "serve_offline"])

In [19]:
# feast_offline_server_process.terminate()

In [24]:
!ps -ef | grep 'serve_offline'
!kill -9 788

1001210+    1369     949  0 15:30 pts/1    00:00:00 /usr/bin/sh -c ps -ef | grep 'serve_offline'
1001210+    1371    1369  0 15:30 pts/1    00:00:00 grep serve_offline
/usr/bin/sh: line 1: kill: (788) - No such process


In [25]:
store = FeatureStore("feast_fraud/client")
store.list_entities()

2024-09-25 15:30:59,804 - Registry cache expired, so refreshing
2024-09-25 15:30:59,805 - Registry cache expired, so refreshing


[]

In [43]:
datetimes = xtrain['ts'].dt.to_pydatetime().tolist()
datetimes=datetimes[:10]
len(datetimes)
# del xtrain
# del xval
# del xtest

/tmp/ipykernel_949/2296142660.py:1: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  datetimes = xtrain['ts'].dt.to_pydatetime().tolist()


10

In [44]:
# datetimes=datetimes[:5]
# len(datetimes)

In [45]:
# Fetch historical data
entity_df = pd.DataFrame.from_dict(
    {
        # "customer_id": [1] * len(datetimes),
        "event_timestamp": datetimes,
    }
)

In [46]:
# fv = store.registry.get_feature_view('training', store.project)
# fv

In [47]:
# %%timeit -r 1 -n 1
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "training:distance_from_last_transaction",
        "training:ratio_to_median_purchase_price",
        "training:used_chip",
        "training:used_pin_number",
        "training:online_order",
        "training:fraud",
    ]
).to_df()
training_df.head()

2024-09-25 15:41:30,306 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-09-25 15:41:30,306 - Registry cache expired, so refreshing
2024-09-25 15:41:30,311 - Registry cache expired, so refreshing
2024-09-25 15:41:30,311 - Registry cache expired, so refreshing
2024-09-25 15:41:30,313 - Connecting FlightClient at grpc://127.0.0.1:8815


,event_timestamp,distance_from_last_transaction,ratio_to_median_purchase_price,used_chip,used_pin_number,online_order,fraud
0,1956-04-15 16:29:54+00:00,0.526379,1.178682,0.0,0.0,1.0,0.0
1,1956-04-15 17:29:54+00:00,0.687956,0.081721,1.0,0.0,1.0,0.0
2,1956-04-15 18:29:54+00:00,8.439479,0.316733,1.0,0.0,1.0,0.0
3,1956-04-15 19:29:54+00:00,0.527061,1.748520,0.0,0.0,1.0,1.0
4,1956-04-15 20:29:54+00:00,0.448277,1.373413,0.0,0.0,1.0,0.0


In [48]:
training_df

,event_timestamp,distance_from_last_transaction,ratio_to_median_purchase_price,used_chip,used_pin_number,online_order,fraud
0,1956-04-15 16:29:54+00:00,0.526379,1.178682,0.0,0.0,1.0,0.0
1,1956-04-15 17:29:54+00:00,0.687956,0.081721,1.0,0.0,1.0,0.0
2,1956-04-15 18:29:54+00:00,8.439479,0.316733,1.0,0.0,1.0,0.0
3,1956-04-15 19:29:54+00:00,0.527061,1.748520,0.0,0.0,1.0,1.0
4,1956-04-15 20:29:54+00:00,0.448277,1.373413,0.0,0.0,1.0,0.0
5,1956-04-15 21:29:54+00:00,0.204930,0.361615,0.0,0.0,1.0,0.0
6,1956-04-15 22:29:54+00:00,3.602882,1.024702,0.0,0.0,0.0,0.0
7,1956-04-15 23:29:54+00:00,3.294577,0.248534,0.0,1.0,1.0,0.0
8,1956-04-16 00:29:54+00:00,42.511988,1.115370,0.0,1.0,1.0,0.0
9,1956-04-16 01:29:54+00:00,0.372917,0.470870,0.0,0.0,0.0,0.0


In [ ]:
!find . -name train.parquet
!pwd

In [ ]:
# Set the input (X) and output (Y) data. 
# The only output data is whether it's fraudulent. All other fields are inputs to the model.

feature_indexes = [
    1,  # distance_from_last_transaction
    2,  # ratio_to_median_purchase_price
    4,  # used_chip
    5,  # used_pin_number
    6,  # online_order
]

label_indexes = [
    7  # fraud
]

X_train =apd.read_csv('data/train.csv')
y_train = X_train.iloc[:, label_indexes]
X_train = X_train.iloc[:, feature_indexes]

X_val = pd.read_csv('data/validate.csv')
y_val = X_val.iloc[:, label_indexes]
X_val = X_val.iloc[:, feature_indexes]

X_test = pd.read_csv('data/test.csv')
y_test = X_test.iloc[:, label_indexes]
X_test = X_test.iloc[:, feature_indexes]


# Scale the data to remove mean and have unit variance. The data will be between -1 and 1, which makes it a lot easier for the model to learn than random (and potentially large) values.
# It is important to only fit the scaler to the training data, otherwise you are leaking information about the global distribution of variables (which is influenced by the test set) into the training set.

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train.values)

Path("artifact").mkdir(parents=True, exist_ok=True)
with open("artifact/test_data.pkl", "wb") as handle:
    pickle.dump((X_test, y_test), handle)
with open("artifact/scaler.pkl", "wb") as handle:
    pickle.dump(scaler, handle)

# Since the dataset is unbalanced (it has many more non-fraud transactions than fraudulent ones), set a class weight to weight the few fraudulent transactions higher than the many non-fraud transactions.
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.values.ravel())
class_weights = {i : class_weights[i] for i in range(len(class_weights))}

## Build the model

The model is a simple, fully-connected, deep neural network, containing three hidden layers and one output layer.

In [ ]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = len(feature_indexes)))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

## Train the model

Training a model is often the most time-consuming part of the machine learning process.  Large models can take multiple GPUs for days.  Expect the training on CPU for this very simple model to take a minute or more.

In [ ]:
# Train the model and get performance
import os
import time

start = time.time()
epochs = 2
history = model.fit(X_train, y_train, epochs=epochs, \
                    validation_data=(scaler.transform(X_val.values),y_val), \
                    verbose = True, class_weight = class_weights)
end = time.time()
print(f"Training of model is complete. Took {end-start} seconds")

## Save the model file

In [ ]:
# Save the model as ONNX for easy use of ModelMesh
model_proto, _ = tf2onnx.convert.from_keras(model)
os.makedirs("models/fraud/1", exist_ok=True)
onnx.save(model_proto, "models/fraud/1/model.onnx")

The output might include TensorFlow messages related to GPUs. You can ignore these messages.

## Confirm the model file was created successfully

The output should include the model name, size, and date. 

In [ ]:
! ls -alRh ./models/

## Test the model

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pickle
import onnxruntime as rt

Load the test data and scaler:

In [ ]:
with open('artifact/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)
with open('artifact/test_data.pkl', 'rb') as handle:
    (X_test, y_test) = pickle.load(handle)

Create an ONNX inference runtime session and predict values for all test inputs:

In [ ]:
sess = rt.InferenceSession("models/fraud/1/model.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: scaler.transform(X_test.values).astype(np.float32)}) 
y_pred_temp = np.asarray(np.squeeze(y_pred_temp[0]))
threshold = 0.95
y_pred = np.where(y_pred_temp > threshold, 1, 0)

Show the results:

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np

y_test_arr = y_test.to_numpy().squeeze()
correct = np.equal(y_pred, y_test_arr).sum().item()
acc = (correct / len(y_pred)) * 100
precision = precision_score(y_test_arr, np.round(y_pred))
recall = recall_score(y_test_arr, np.round(y_pred))

print(f"Eval Metrics: \n Accuracy: {acc:>0.1f}%, "
      f"Precision: {precision:.4f}, Recall: {recall:.4f} \n")

c_matrix = confusion_matrix(y_test_arr, y_pred)
ConfusionMatrixDisplay(c_matrix).plot()

## Example: Is Sally's transaction likely to be fraudulent?

Here is the order of the fields from Sally's transaction details:
* distance_from_last_transaction
* ratio_to_median_price
* used_chip 
* used_pin_number
* online_order 

In [ ]:
sally_transaction_details = [
    [0.3111400080477545,
    1.9459399775518593, 
    1.0, 
    0.0, 
    0.0]
    ]
prediction = sess.run([output_name], {input_name: scaler.transform(sally_transaction_details).astype(np.float32)})

print("Is Sally's transaction predicted to be fraudulent? (true = YES, false = NO) ")
print(np.squeeze(prediction) > threshold)

print("How likely was Sally's transaction to be fraudulent? ")
print("{:.5f}".format(np.squeeze(prediction)) + "%")